In [122]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [3]:
train_X = pd.read_csv('X_train_knn.csv')
test_X = pd.read_csv('X_test_knn.csv')
train_y = pd.read_csv('y_train.csv')
test_y = pd.read_csv('y_test.csv')

In [18]:
scaler = StandardScaler()
nor_train = scaler.fit_transform(train_X)
nor_test = scaler.transform(test_X)
xgb_weight = [1 if i==0 else 10 for i in train_y['TARGET'].to_numpy()]

In [180]:
train_X

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,...,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max,DAY_DIFF,PAY_INSTALL_PROP,LIVING_AVG,LIVING_MISSING,SOCIAL_MISSING,FLAG_DOCUMENT_CNT
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,14.083333,1.342883,7.000000,46.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.394444,0.976164,0.213333,0.0,0.0,2.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,15.750000,0.299998,0.000000,46.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,12.545455,0.333333,0.290079,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.333333,1.000000,0.000000,46.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,21.200000,0.750000,2.000000,46.0,0.0,1.0
79996,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,48.088889,0.916667,0.341218,3.0,0.0,1.0
79997,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,38.388348,0.818155,0.000000,46.0,0.0,1.0
79998,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,20.533597,0.847613,0.233177,22.0,0.0,2.0


XGBoost after Grid Search

In [5]:
xgb_grid = XGBClassifier(max_depth = 2, n_estimators= 1000, 
                         reg_alpha = 10,subsample= 0.6,learning_rate=0.05)
xgb_grid.fit(train_X, train_y['TARGET'], sample_weight=xgb_weight)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [6]:
# grid xgb train
xgb_grid_train_pred = xgb_grid.predict(train_X)
roc_auc_score(train_y['TARGET'],xgb_grid_train_pred)

0.7280247241596663

In [7]:
# grid xgb test 
xgb_grid_test_pred = xgb_grid.predict(test_X)
roc_auc_score(test_y['TARGET'],xgb_grid_test_pred)

0.6992029657482723

Gradient Boosting

In [34]:
gdc = GradientBoostingClassifier(learning_rate=0.03, n_estimators=650, 
                                 subsample=0.65)
gdc.fit(train_X,train_y['TARGET'],sample_weight=xgb_weight)

GradientBoostingClassifier(learning_rate=0.03, n_estimators=650, subsample=0.65)

In [35]:
# grid gdc train 
gdc_train_pred = gdc.predict(train_X)
roc_auc_score(train_y['TARGET'], gdc_train_pred)

0.7340861505526759

In [36]:
# grid gdc test
gdc_test_pred = gdc.predict(test_X)
roc_auc_score(test_y['TARGET'], gdc_test_pred)

0.6982242468051445

Logistic Regression

In [12]:
logic = LogisticRegression(penalty='l1',solver='saga',
                           class_weight={0:1,1:11})
logic.fit(nor_train,train_y['TARGET'])

/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(class_weight={0: 1, 1: 11}, penalty='l1', solver='saga')

In [32]:
# log train 
log_train_pred = logic.predict(nor_train)
roc_auc_score(train_y['TARGET'],log_train_pred)

0.6998133576921611

In [33]:
# log test
log_test_pred = logic.predict(nor_test)
roc_auc_score(test_y['TARGET'],log_test_pred)

0.6945913733475135

Linear SVC

In [19]:
li_svc_grid = LinearSVC(C = 0.01,class_weight={0:1,1:11},tol=0.01,
                        max_iter=800)
li_svc_grid.fit(nor_train,train_y['TARGET'])

/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=0.01, class_weight={0: 1, 1: 11}, max_iter=800, tol=0.01)

In [21]:
# linear svc train 
li_svc_grid_train_pred = li_svc_grid.predict(nor_train)
roc_auc_score(train_y['TARGET'],li_svc_grid_train_pred)

0.7006409455254997

In [22]:
# linear svc test
li_svc_grid_test_pred = li_svc_grid.predict(nor_test)
roc_auc_score(test_y['TARGET'],li_svc_grid_test_pred)

0.6966600568742268

Hist Gradient Boosting

In [29]:
hgb = HistGradientBoostingClassifier(learning_rate=0.08,max_iter=800,
                                     max_depth=3,scoring='roc_auc',
                                     interaction_cst='pairwise',
                                     tol=0.0003,class_weight={0:1,1:11},
                                     l2_regularization=5)
hgb.fit(train_X,train_y['TARGET'])

HistGradientBoostingClassifier(class_weight={0: 1, 1: 11},
                               interaction_cst='pairwise', l2_regularization=5,
                               learning_rate=0.08, max_depth=3, max_iter=800,
                               scoring='roc_auc', tol=0.0003)

In [30]:
# hgb rdf train
hgb_train_pred = hgb.predict(train_X)
roc_auc_score(train_y['TARGET'],hgb_train_pred)

0.7086009856531948

In [31]:
# hgb rdf test
hgb_test_pred = hgb.predict(test_X)
roc_auc_score(test_y['TARGET'],hgb_test_pred)

0.6944882288644649

Random Forest

In [44]:
rdf = RandomForestClassifier(max_depth = 7, n_estimators=1100,
                             class_weight={0:1,1:11})
rdf.fit(train_X,train_y['TARGET'])

RandomForestClassifier(class_weight={0: 1, 1: 11}, max_depth=7,
                       n_estimators=1100)

In [45]:
# random forest train 
rdf_train_pred=rdf.predict(train_X)
roc_auc_score(train_y['TARGET'],rdf_train_pred)

0.7263525827037565

In [46]:
# random forest test 
rdf_test_pred=rdf.predict(test_X)
roc_auc_score(test_y['TARGET'],rdf_test_pred)

0.6708454111506587

Random Forest Feature Selection

In [47]:
rdf_features = pd.DataFrame({'feature':rdf.feature_names_in_,
                             'importance':rdf.feature_importances_})
rdf_features

,feature,importance
0,NAME_CONTRACT_TYPE_Cash loans,0.001425
1,NAME_CONTRACT_TYPE_Revolving loans,0.001531
2,CODE_GENDER_F,0.008222
3,CODE_GENDER_M,0.007809
4,CODE_GENDER_XNA,0.000000
...,...,...
480,PAY_INSTALL_PROP,0.028292
481,LIVING_AVG,0.002263
482,LIVING_MISSING,0.004416
483,SOCIAL_MISSING,0.000000


In [52]:
rdf_features_arr = rdf_features.sort_values(by='importance',ascending=False)['feature'].to_numpy()[:300]

In [54]:
rfd_train = train_X[rdf_features_arr]
rfd_test = test_X[rdf_features_arr]

In [55]:
scaler = StandardScaler()
nor_rdf_train = scaler.fit_transform(rfd_train)
nor_rdf_test = scaler.transform(rfd_test)

RDF + XGB

In [98]:
xgb = XGBClassifier(max_depth = 2, n_estimators= 1000, 
                    reg_alpha = 7,subsample= 0.6)
xgb.fit(rfd_train, train_y['TARGET'], sample_weight=xgb_weight)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [99]:
# xgb rdf train
xgb_rdf_train_pred = xgb.predict(rfd_train)
roc_auc_score(train_y['TARGET'],xgb_rdf_train_pred)

0.8093560328051035

In [100]:
# xgb rdf test
xgb_rdf_test_pred =xgb.predict(rfd_test)
roc_auc_score(test_y['TARGET'],xgb_rdf_test_pred)

0.6772856945626433

In [ ]:
0.6790509435985458

RDF + HGB

In [95]:
hgb = HistGradientBoostingClassifier(learning_rate=0.1,max_iter=1000,
                                     max_depth=4,scoring='roc_auc',
                                     interaction_cst='pairwise',
                                     tol=0.001,class_weight={0:1,1:11},
                                     l2_regularization=5)
hgb.fit(rfd_train,train_y['TARGET'])

HistGradientBoostingClassifier(class_weight={0: 1, 1: 11},
                               interaction_cst='pairwise', l2_regularization=5,
                               max_depth=4, max_iter=1000, scoring='roc_auc',
                               tol=0.001)

In [96]:
# hgb rdf train
hgb_rdf_train_pred = hgb.predict(rfd_train)
roc_auc_score(train_y['TARGET'],hgb_rdf_train_pred)

0.7154875636468181

In [97]:
# hgb rdf test
hgb_rdf_test_pred = hgb.predict(rfd_test)
roc_auc_score(test_y['TARGET'],hgb_rdf_test_pred)

0.6900322846504705

In [ ]:
0.692954084998369 4 1000

In [105]:
li_svc_grid = LinearSVC(C = 0.01,class_weight={0:1,1:11},tol=0.001,
                        max_iter=800)
li_svc_grid.fit(nor_rdf_train,train_y['TARGET'])

/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=0.01, class_weight={0: 1, 1: 11}, max_iter=800, tol=0.001)

In [106]:
# linear svc train 
li_svc_rdf_train_pred = li_svc_grid.predict(nor_rdf_train)
roc_auc_score(train_y['TARGET'],li_svc_rdf_train_pred)

0.6981211707609936

In [107]:
# linear svc test
li_svc_rdf_test_pred = li_svc_grid.predict(nor_rdf_test)
roc_auc_score(test_y['TARGET'],li_svc_rdf_test_pred)

0.6956711516656645

Combining models (Manual Stacking; doesn't work well) 

In [82]:
grace_train = pd.read_csv('model_data/grace_train_pred.csv')
grace_test = pd.read_csv('model_data/grace_train_pred.csv')
sally_train = pd.read_csv('model_data/y_train_pred.csv')
sally_test = pd.read_csv('model_data/y_test_pred.csv')

In [113]:
total_train_pred = pd.DataFrame(list(map(list,zip(xgb_grid_train_pred,gdc_train_pred,log_train_pred,
                                                  li_svc_grid_train_pred,hgb_train_pred,hgb_rdf_train_pred,
                                                  li_svc_rdf_train_pred))),
                                columns =['xgb','gdc','log','lsvc','hgb','hgb_rdf','li_svc_rdf'])
                               
total_test_pred = pd.DataFrame(list(map(list,zip(xgb_grid_test_pred,gdc_test_pred,log_test_pred,
                                                  li_svc_grid_test_pred,hgb_test_pred,hgb_rdf_test_pred,
                                                  li_svc_rdf_test_pred))),
                                columns =['xgb','gdc','log','lsvc','hgb','hgb_rdf','li_svc_rdf'])                  

In [116]:
total_train_pred=total_train_pred.merge(sally_train,left_index=True,right_index=True)
total_test_pred=total_test_pred.merge(sally_test,left_index=True,right_index=True)
total_train_pred=total_train_pred.merge(grace_train,left_index=True,right_index=True)
total_test_pred=total_test_pred.merge(grace_test,left_index=True,right_index=True)

In [124]:
total_train, total_val, y_total_train, y_total_val = train_test_split(total_train_pred,train_y['TARGET'],test_size=0.2, random_state=42)


In [126]:
logic_final = LogisticRegression(penalty='l1',solver='saga',class_weight={0:1,1:11})
logic_final.fit(total_train,y_total_train)

/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(class_weight={0: 1, 1: 11}, penalty='l1', solver='saga')

In [128]:
roc_auc_score(y_total_train,logic_final.predict(total_train))

0.7853177017416664

In [127]:
roc_auc_score(y_total_val,logic_final.predict(total_val))

0.7830984230840657

In [129]:
roc_auc_score(test_y['TARGET'],logic_final.predict(total_test_pred))

0.6354453358338222

In [182]:
# 
roc_auc_score(test_y['TARGET'],(total_test_pred.mean(axis=1))>0.4)

0.6980714699146771

In [184]:
#np.percentile(92)
roc_auc_score(test_y['TARGET'],total_test_pred.mean(axis=1) > roc_auc_score(test_y['TARGET'],total_test_pred.quantile(axis=1,q=0.92)))

0.6814883494247274

In [160]:
li_svc_final = LinearSVC(C = 0.01,tol=0.001,
                        max_iter=800)
li_svc_final.fit(total_train,y_total_train)

LinearSVC(C=0.01, max_iter=800, tol=0.001)

In [173]:
param_grid = {
    'C': [0.1, 0.5,1],
    'tol': [0.01, 0.1, 0.05],
    'max_iter':[500,700],
    #'class_weight': [{0:1, 1:9}]
}
li_svc_grid = LinearSVC(class_weight={0:1, 1:11})
grid_search = GridSearchCV(li_svc_grid, param_grid, cv=4, verbose=2, scoring='roc_auc')
grid_search.fit(total_train,y_total_train)

Fitting 4 folds for each of 18 candidates, totalling 72 fits
[CV] END ......................C=0.1, max_iter=500, tol=0.01; total time=   0.3s
[CV] END ......................C=0.1, max_iter=500, tol=0.01; total time=   0.3s
[CV] END ......................C=0.1, max_iter=500, tol=0.01; total time=   0.3s
[CV] END ......................C=0.1, max_iter=500, tol=0.01; total time=   0.2s
[CV] END .......................C=0.1, max_iter=500, tol=0.1; total time=   0.1s
[CV] END .......................C=0.1, max_iter=500, tol=0.1; total time=   0.1s
[CV] END .......................C=0.1, max_iter=500, tol=0.1; total time=   0.2s
[CV] END .......................C=0.1, max_iter=500, tol=0.1; total time=   0.1s
[CV] END ......................C=0.1, max_iter=500, tol=0.05; total time=   0.2s
[CV] END ......................C=0.1, max_iter=500, tol=0.05; total time=   0.2s
[CV] END ......................C=0.1, max_iter=500, tol=0.05; total time=   0.2s
[CV] END ......................C=0.1, max_iter=5

/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ......................C=0.5, max_iter=500, tol=0.01; total time=   0.9s


/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ......................C=0.5, max_iter=500, tol=0.01; total time=   0.9s


...

[CV] END ........................C=1, max_iter=700, tol=0.05; total time=   1.2s


/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ........................C=1, max_iter=700, tol=0.05; total time=   1.2s


/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


GridSearchCV(cv=4, estimator=LinearSVC(class_weight={0: 1, 1: 11}),
             param_grid={'C': [0.1, 0.5, 1], 'max_iter': [500, 700],
                         'tol': [0.01, 0.1, 0.05]},
             scoring='roc_auc', verbose=2)

In [174]:
grid_search.best_params_

{'C': 1, 'max_iter': 500, 'tol': 0.1}

In [177]:
li_svc_grid = LinearSVC(class_weight={0:1, 1:11},C=1,max_iter=500,tol=0.1)
li_svc_grid.fit(total_train_pred,train_y)

/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arianeyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=1, class_weight={0: 1, 1: 11}, max_iter=500, tol=0.1)

In [185]:
roc_auc_score(train_y['TARGET'],li_svc_grid.predict(total_train_pred))

0.7961906597339174

In [178]:
roc_auc_score(test_y['TARGET'],li_svc_grid.predict(total_test_pred))

0.5208640694492741

In [186]:
param_grid = {
    'max_depth':[3,5,8],
    'n_estimators':[1000,400,200]
}
rdf = RandomForestClassifier(class_weight={0:1,1:11})
grid_search = GridSearchCV(rdf, param_grid, cv=4, verbose=2, scoring='roc_auc')
grid_search.fit(total_train_pred,train_y['TARGET'])

Fitting 4 folds for each of 9 candidates, totalling 36 fits
[CV] END .....................max_depth=3, n_estimators=1000; total time=   7.0s
[CV] END .....................max_depth=3, n_estimators=1000; total time=   7.0s
[CV] END .....................max_depth=3, n_estimators=1000; total time=   6.9s
[CV] END .....................max_depth=3, n_estimators=1000; total time=   6.9s
[CV] END ......................max_depth=3, n_estimators=400; total time=   2.8s
[CV] END ......................max_depth=3, n_estimators=400; total time=   2.8s
[CV] END ......................max_depth=3, n_estimators=400; total time=   2.8s
[CV] END ......................max_depth=3, n_estimators=400; total time=   2.8s
[CV] END ......................max_depth=3, n_estimators=200; total time=   1.4s
[CV] END ......................max_depth=3, n_estimators=200; total time=   1.4s
[CV] END ......................max_depth=3, n_estimators=200; total time=   1.4s
[CV] END ......................max_depth=3, n_est

GridSearchCV(cv=4, estimator=RandomForestClassifier(class_weight={0: 1, 1: 11}),
             param_grid={'max_depth': [3, 5, 8],
                         'n_estimators': [1000, 400, 200]},
             scoring='roc_auc', verbose=2)

In [187]:
grid_search.best_params_

{'max_depth': 5, 'n_estimators': 1000}

In [188]:
rfc = RandomForestClassifier(class_weight={0:1,1:11},max_depth=5,n_estimators=1000)
rfc.fit(total_train_pred,train_y)

/var/folders/w1/vc3f0wy917q9_qnbxvz9gvfh0000gn/T/ipykernel_37409/656547043.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(total_train_pred,train_y)


RandomForestClassifier(class_weight={0: 1, 1: 11}, max_depth=5,
                       n_estimators=1000)

In [189]:
roc_auc_score(train_y['TARGET'],rfc.predict(total_train_pred))

0.8001307190195454

In [190]:
roc_auc_score(test_y['TARGET'],rfc.predict(total_test_pred))

0.49426808931194477